In [ ]:
#|default_exp engine

# Engine

> Game engine, main simulator

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|export
from fastcore.all import *
import polvo as pv
import truco as tr
import numpy as np
import random
from copy import deepcopy
from collections import deque

/home/lgvaz/miniconda3/envs/truco/lib/python3.10/site-packages/polvo/utils/logging.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
#|export
class Player:
    def __init__(self, team): 
        self.team, self.hand = team, []
        self.team_sign = dict(t1=1, t2=-1)[team]
    def new_hand(self, hand): self.hand = hand
    def __repr__(self): return f'Player {self.team}: {self.hand}'
    def play_card(self, turn_cards): 
        ...

In [ ]:
#|export
class HumanPlayer(Player):
    def play_card(self, turn_cards):
        card = input(f'Table: {turn_cards} | Hand: {self.hand}')
        self.hand.remove(card)
        return card

In [ ]:
#|export
class BotPlayer(Player):
    def play_card(self, turn_cards):
        return self.hand.pop(0)

In [ ]:
#|export
class FixedOrderBotPlayer(Player):
    def __init__(self, team, order):
        self.order = np.array(order)
        super().__init__(team=team)
        
    def play_card(self, turn_cards):
        idx, self.order = self.order[0], self.order[1:]
        self.order[self.order>idx] -= 1 # Adjust index for cards that were after current one being popped
        return self.hand.pop(idx)

In [ ]:
bot = FixedOrderBotPlayer('t1', [2, 1, 0])
bot.new_hand('qs js ks'.split())
test_eq(bot.play_card([]), 'ks')
test_eq(bot.play_card([]), 'js')
test_eq(bot.play_card([]), 'qs')

In [ ]:
bot = FixedOrderBotPlayer('t1', [1, 2, 0])
bot.new_hand('qs js ks'.split())
test_eq(bot.play_card([]), 'js')
test_eq(bot.play_card([]), 'ks')
test_eq(bot.play_card([]), 'qs')

In [ ]:
#|export
def turn_finish(turn_cards, player_order, round_rank):
    ranks = tr.cards2ranks(turn_cards, round_rank)
    # necessary to get 0 if two cards of the same rank are played
    winners_mask = ranks==ranks.max()
    turn_score = sum(set([p.team_sign for p, w in zip (player_order, winners_mask) if w]))

    rotation = np.argwhere(winners_mask).flatten()[-1]
    if sum(winners_mask)>1: rotation += 1
    
    return turn_score, -rotation

In [ ]:
p11, p12 = FixedOrderBotPlayer('t1', [1, 0, 2]), BotPlayer('t1')
p21, p22 = BotPlayer('t2'), FixedOrderBotPlayer('t2', [2, 1, 0])

In [ ]:
faceup = '4d'

In [ ]:
p11.new_hand('5d 3h qs'.split())
p12.new_hand('6s 6h 4s'.split())

In [ ]:
round_rank = tr.round_rank(faceup)
round_card_pool = tr.round_card_pool(p11.hand+p12.hand, faceup)

In [ ]:
t2_hand = random.sample(round_card_pool, 6)
p21.new_hand(t2_hand[:3])
p22.new_hand(t2_hand[3:])

In [ ]:
r = turn_finish('3h 3s 4s 7c'.split(), [p11, p21, p12, p22], tr.round_rank('5c'))
test_eq(r, (0, -2))

In [ ]:
r = turn_finish('3h as 4s 7c'.split(), [p11, p21, p12, p22], tr.round_rank('5c'))
test_eq(r, (1, 0))

In [ ]:
r = turn_finish('ah 2s 4s 7c'.split(), [p11, p21, p12, p22], tr.round_rank('5c'))
test_eq(r, (-1, -1))

In [ ]:
r = turn_finish('ah 2s 6s 6c'.split(), [p11, p21, p12, p22], tr.round_rank('5c'))
test_eq(r, (-1, -3))

In [ ]:
#|export
def calculate_round_score(turn_scores):
    ts = np.asarray(turn_scores)
    nonzero_idx = np.nonzero(ts)[0]
    if nonzero_idx.size > 0: ts[ts==0] = ts[nonzero_idx[0]]
    return sum(ts)

In [ ]:
test_eq(calculate_round_score([0, -1, 1]), -1)
test_eq(calculate_round_score([-1, -1, 1]), -1)
test_eq(calculate_round_score([1, 0, -1]), 1)
test_eq(calculate_round_score([1, 0, 1]), 3)

In [ ]:
#|export
# TODO: I want to remove class methods and make it functional
class RoundManager:
    def __init__(self, p11, p12, p21, p22):
        self.p11, self.p12, self.p21, self.p22 = map(deepcopy, (p11, p12, p21, p22))
        
        # TODO: Dynamic
        self.player_order = deque([self.p11, self.p21, self.p12, self.p22])
        
    def new_round(self):
        self.score = []
        self.history = []
        
    def play_turn(self, round_rank):
        turn_cards = {}
        for p in self.player_order:
            card = p.play_card(turn_cards)
            turn_cards[p] = card
            self.history.append((p.team, card))
        turn_score, rotation = turn_finish(turn_cards.values(), self.player_order, round_rank)
        self.score.append(turn_score)
        self.player_order.rotate(rotation)
            
    def play_round(self, round_rank):
        self.new_round()
        for _ in range(3): self.play_turn(round_rank)
        return calculate_round_score(self.score)

In [ ]:
rm = RoundManager(p11, p12, p21, p22)
rm.play_round(round_rank)

1

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()